In [9]:
from tkinter import *
from tkinter import ttk,messagebox
from tkinter import filedialog
from PIL import ImageTk, Image
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow import keras
from tensorflow.keras.layers import GlobalAveragePooling2D, BatchNormalization, Dropout,AveragePooling2D
import tensorflow as tf
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import *
from tensorflow import keras
from tensorflow.keras import layers
from reportlab.pdfgen.canvas import Canvas
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import mm
from reportlab.pdfbase.pdfmetrics import registerFontFamily
from reportlab.pdfbase.ttfonts import TTFont
from datetime import date
import numpy as np
import cv2
from numpy import asarray
from textwrap import wrap

def load_knee_Model():
    base = VGG19(weights='imagenet', include_top=False,input_tensor=Input(shape=(150, 150, 3)))

    # Take output from pre-trained and put through custom classifier
    x = base.output
    x = tf.keras.layers.GlobalMaxPooling2D()(x)
    x =tf.keras.layers.Dense(1024, activation='relu')(x)
    x =tf.keras.layers.Dense(512, activation='relu')(x)
    x =tf.keras.layers.Dense(64, activation='relu')(x)
    output = tf.keras.layers.Dense(5, activation='softmax')(x) #OUtput is an array of probabilities 0-1

    model = Model(inputs=base.input, outputs=output)
    for layer in model.layers[:11]:
        layer.trainable = False
    for layer in model.layers[11:]:
        layer.trainable = True

    # we need to recompile the model for these modifications to take effect
    # we use SGD with a low learning rate
    from tensorflow.keras.optimizers import SGD, Adam
    model.compile(optimizer=SGD(learning_rate=0.001), loss='categorical_crossentropy', metrics=['acc'])
    model = keras.models.load_model('VGG19 Final.h5')
    return model

def knee_predict(x):
    model = load_knee_Model()
    img = image.load_img(x, target_size=(150,150))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    x_processed = preprocess_input(x)
    return model.predict(x_processed)


def load_Brain_model():
    vgg = VGG16(input_shape = [224,224] + [3], weights='imagenet', include_top=False)
    for layer in vgg.layers:
        layer.trainable = False
    x = Flatten()(vgg.output)
    prediction = Dense(2, activation='softmax')(x)
    model = Model(inputs=vgg.input, outputs=prediction)
    model.compile(
         loss='categorical_crossentropy',
         optimizer='adam',
         metrics=['accuracy']
    )
    MODEL = keras.models.load_model("C:/Users/shahe/AI PROJECTS/Brain Tumor Detection/Brain Tumor Detection.h5")
    return MODEL

def Brain_Tumor_predict(img):
    model = load_Brain_model()
    img_normal = image.load_img(img, target_size = (224, 224))
    x = image.img_to_array(img_normal)
    x = np.expand_dims(x, axis = 0)
    x_processed = preprocess_input(x)
    return model.predict(x_processed)

def load_chest_model():
    vgg = VGG16(input_shape = [224,224] + [3], weights='imagenet', include_top=False)
    for layer in vgg.layers:
        layer.trainable = False
    x = Flatten()(vgg.output)
    prediction = Dense(2, activation='softmax')(x)
    model = Model(inputs=vgg.input, outputs=prediction)
    model.compile(
         loss='categorical_crossentropy',
         optimizer='adam',
         metrics=['accuracy']
    )
    MODEL = keras.models.load_model("covid_vgg1.h5")
    return MODEL

def chest_predict(img):
    model = load_chest_model()
    img_normal = image.load_img(img, target_size = (224, 224))
    x = image.img_to_array(img_normal)
    x = np.expand_dims(x, axis = 0)
    x_processed = preprocess_input(x)
    return model.predict(x_processed)

def load_alzheimers_model():
    layers_unlocked = True
    vg_model = tf.keras.applications.vgg16.VGG16(include_top=False,input_shape=(176,206,3), pooling = 'max') 
    vg_model.get_layer('block1_conv1').trainable = layers_unlocked
    vg_model.get_layer('block1_conv2').trainable = layers_unlocked
    vg_model.get_layer('block2_conv1').trainable = layers_unlocked
    vg_model.get_layer('block2_conv2').trainable = layers_unlocked
    vg_model.get_layer('block3_conv1').trainable = layers_unlocked
    vg_model.get_layer('block3_conv2').trainable = layers_unlocked
    vg_model.get_layer('block3_conv3').trainable = layers_unlocked
    vg_model.get_layer('block4_conv1').trainable = layers_unlocked
    vg_model.get_layer('block4_conv2').trainable = layers_unlocked
    vg_model.get_layer('block4_conv3').trainable = layers_unlocked
    flat = Flatten()(vg_model.output)
    fc1 = Dense(1024,activation='relu', kernel_initializer='he_uniform')(flat) 
    dp1 = Dropout(0.25)(fc1)                                                   
    output = Dense(4,activation='softmax')(dp1)                                
    vg_model = Model(inputs=vg_model.inputs, outputs=output)
    opt = tf.keras.optimizers.SGD(learning_rate=0.0001, momentum=0.9, nesterov=True,name='SGD') 
    vg_model.compile(optimizer=opt, loss=tf.keras.losses.categorical_crossentropy, metrics=['accuracy'])
    MODEL = keras.models.load_model("C:/Users/shahe/Desktop/Alzheimers.h5")
    return MODEL

def alzheimers_predict(img):
    model = load_alzheimers_model()
    Img = image.load_img(img, target_size = (176, 208))
    x = image.img_to_array(Img)
    x = np.expand_dims(x, axis = 0)
    x_processed = preprocess_input(x)
    return model.predict(x_processed)
    

def download_model():
    model = Sequential()

    conv_base = DenseNet201(input_shape=(224,224,3), include_top=False, pooling='max',weights='imagenet')
    model.add(conv_base)
    model.add(BatchNormalization())
    model.add(Dense(2048, activation='relu', kernel_regularizer=l1_l2(0.01)))
    model.add(BatchNormalization())
    model.add(Dense(8, activation='softmax'))

    train_layers = [layer for layer in conv_base.layers[::-1][:5]]

    for layer in conv_base.layers:
        if layer in train_layers:
            layer.trainable = True
    model.save("C:/Users/shahe/AI PROJECTS/Breast Cancer Mammography/model/model.h5")
    
def load_model():
    download_model()
    model=tf.keras.models.load_model("C:/Users/shahe/AI PROJECTS/Breast Cancer Mammography/model/model.h5", compile=False)
    model.compile(optimizer =tf.keras.optimizers.Adam(learning_rate=0.00001,decay=0.0001),metrics=["accuracy"],
                  loss= tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1))
    model.load_weights("C:/Users/shahe/Desktop/MED-SMART STARTUP PLAN/models/mammography.h5")
    return model

def preprocess(image):
    kernel = np.array([[0,-1,0], [-1,5,-1], [0,-1,0]])
    im = cv2.filter2D(image, -1, kernel)
    return im

def predict_img(img):
    model = load_model()
    class_name=['Benign with Density=1','Malignant with Density=1',
            'Benign with Density=2','Malignant with Density=2',
            'Benign with Density=3','Malignant with Density=3',
            'Benign with Density=4','Malignant with Density=4']
    img = Image.open(x)
    img = asarray(img)
    img=preprocess(img)
    img=img/255.0
    im=img.reshape(-1,224,224,3)
    pred=model.predict(im)[0]
    return [float(pred[i]) for i in range(8)]

def load_Lung_model():
    vgg = VGG16(input_shape = [224,224] + [3], weights='imagenet', include_top=False)
    for layer in vgg.layers:
        layer.trainable = False
    x = Flatten()(vgg.output)
    prediction = Dense(2, activation='softmax')(x)
    model = Model(inputs=vgg.input, outputs=prediction)
    model.compile(
      loss='categorical_crossentropy',
      optimizer='adam',
      metrics=['accuracy']
    )
    MODEL = keras.models.load_model("C:/Users/shahe/AI PROJECTS/Lung Cancer/Lung Cancer Detection.h5")
    return MODEL

def lung_predict(img):
    model = load_Lung_model()
    img_normal = image.load_img(img, target_size = (224, 224))
    x = image.img_to_array(img_normal)
    x = np.expand_dims(x, axis = 0)
    x_processed = preprocess_input(x)
    return model.predict(x_processed)

def GenerateReport(img, name, Type):
    height, width = A4
    canvas = Canvas(str(name)+ " - " + str(Type) + ".pdf", pagesize=(height,width))

    canvas.drawString(165, 795, "MEDSMART SOLUTIONS REPORT GENERATOR")

    canvas.line(35,785,560,785)

    canvas.drawString(35, 755, "File Number: " + str(x1))
    canvas.drawString(35, 735, "Patient Name: " + str(x2))
    canvas.drawString(35, 715, "Date of Birth: " + str(x3))
    canvas.drawString(35, 695, "Date of Study: " + str(x4))
    canvas.drawString(35, 675, "Date of Report: " + str(x5))
    canvas.drawString(35, 655, "Scan Type: " + str(x6))
    
    if img != "":
        canvas.drawImage(img ,320 ,595 ,width=200, height=185, preserveAspectRatio=True, mask='auto')

    canvas.drawString(200, 575, "X-RAY SCAN TYPE: " + str(x7))

    t = canvas.beginText()
    t.setCharSpace(1)
    t.setTextOrigin(93, 525)
    text = "Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book."
    wraped_text = "\n".join(wrap(x7, 75))
    t.textLines(wraped_text)
    canvas.drawString(35,525,"Findings: ")
    canvas.drawText(t)

    t = canvas.beginText()
    t.setCharSpace(1)
    t.setTextOrigin(120, 458)
    text = "Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book."
    wraped_text = "\n".join(wrap(x8, 67))
    t.textLines(wraped_text)
    canvas.drawString(35,458,"Impressions: ")
    canvas.drawText(t)

    t = canvas.beginText()
    t.setCharSpace(1)
    t.setTextOrigin(185, 379)
    text = "Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book."
    wraped_text = "\n".join(wrap(x9, 56))
    t.textLines(wraped_text)
    canvas.drawString(35,379,"Machine Aided Results: ")
    canvas.drawText(t)

    t = canvas.beginText()
    t.setCharSpace(1)
    t.setTextOrigin(205, 290)
    text = "Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book."
    wraped_text = "\n".join(wrap(x10, 52))
    t.textLines(wraped_text)
    canvas.drawString(35,290,"Human Specialist Results: ")
    canvas.drawText(t)

    t = canvas.beginText()
    t.setCharSpace(1)
    t.setTextOrigin(180, 205)
    text = "Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book."
    wraped_text = "\n".join(wrap(x11, 58))
    t.textLines(wraped_text)
    canvas.drawString(35,205,"Additional Information: ")
    canvas.drawText(t)

    today = date.today()
    d1 = today.strftime("%d/%m/%Y")

    canvas.drawString(440,105,str(d1))
    canvas.drawString(65,95,"-"*45)
    canvas.drawString(410,95,"-"*30)

    canvas.drawString(65,75,"SIGNATURE OF SPECIALIST")
    canvas.drawString(410,75,"DATE AUTHORIZED")

    canvas.line(35,55,560,55)

    canvas.save()


class Windows(object):
    def ReportScreen(self):
        window3 = Toplevel(root)
        window3.geometry("500x650")
        window3.resizable(0,0)

        header = Label(window3, text="MEDSMART X-RAY SOLUTIONS", font=("Arial", 22))
        header.place(x=250, y=55, anchor="center")

        FileNum = Label(window3,text="File Number:", font=("Arial",12))
        FileNum.place(x=125, y=100, anchor="center")
        text1 = Entry(window3)
        text1.place(x=300, y=100, anchor="center")

        Name = Label(window3,text="Patient Name:", font=("Arial",12))
        Name.place(x=125, y=125, anchor="center")
        text2 = Entry(window3)
        text2.place(x=300, y=125, anchor="center")

        DOB = Label(window3,text="Date of Birth:", font=("Arial",12))
        DOB.place(x=125, y=150, anchor="center")
        text3 = Entry(window3)
        text3.place(x=300, y=150, anchor="center")

        DOS = Label(window3,text="Date of Study:", font=("Arial",12))
        DOS.place(x=125, y=175, anchor="center")
        text4 = Entry(window3)
        text4.place(x=300, y=175, anchor="center")

        DOR = Label(window3,text="Date of Report:", font=("Arial",12))
        DOR.place(x=125, y=200, anchor="center")
        text5 = Entry(window3)
        text5.place(x=300, y=200, anchor="center")

        ST = Label(window3,text="Scan Type:", font=("Arial",12))
        ST.place(x=125, y=225, anchor="center")
        text6 = Entry(window3)
        text6.place(x=300, y=225, anchor="center")
        if self.query.get() != "Please Select A Model":
            text6.insert(0, self.query.get())
            text6.config(state=DISABLED)
        
        Title = Label(window3,text="Title of Study:", font=("Arial",12))
        Title.place(x=125, y=250, anchor="center")
        text7 = Entry(window3)
        text7.place(x=300, y=250, anchor="center")

        Findings = Label(window3,text="Findings:", font=("Arial",12))
        Findings.place(x=125, y=280, anchor="center")
        t1 = Text(window3,  height=5, width=25)# added one text box
        t1.place(x=340, y=315, anchor="center")

        Impressions = Label(window3,text="Impressions:", font=("Arial",12))
        Impressions.place(x=125, y=375, anchor="center")
        t2 = Text(window3,  height=5, width=25)# added one text box
        t2.place(x=340, y=405, anchor="center")

        Human = Label(window3,text="Human Study Results:", font=("Arial",12))
        Human.place(x=125, y=470, anchor="center")
        t3 = Text(window3,  height=5, width=25)# added one text box
        t3.place(x=340, y=500, anchor="center")

        Adds = Label(window3,text="Additionals:", font=("Arial",12))
        Adds.place(x=125, y=560, anchor="center")
        t4 = Text(window3,  height=2, width=25)# added one text box
        t4.place(x=340, y=570, anchor="center")
        
        def getInputs():
            global x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11
            x1 = str(text1.get())
            x2 = str(text2.get())
            x3 = str(text3.get())
            x4 = str(text4.get())
            x5 = str(text5.get())
            x6 = str(text6.get())
            x7 = str(text7.get())
            x8 = str(t1.get("1.0",'end-1c'))
            x9 = str(t2.get("1.0",'end-1c'))
            x10 = str(t3.get("1.0",'end-1c'))
            x11 = str(t4.get("1.0",'end-1c'))
            GenerateReport(x,x2,x7)
            messagebox.showinfo("Complete!", "Report Successfully Generated!")
        
        Generate = Button(window3,text="Generate REPORT", command=getInputs)
        Generate.place(x=250, y=610, anchor="center")

    
        
    def SimpleDisplay(self):
        window2 = Toplevel(root)
        window2.geometry("500x500")

        header = Label(window2,text="MEDSMART X-RAY SOLUTIONS", font=("Arial", 22))
        header.place(x=250, y=55, anchor="center")

        Stype = Label(window2, text="Scan Type: " + self.query.get(), font=("Arial", 14))
        Stype.place(x=260, y=125, anchor="center")
        
        img = Image.open(x)
        img = img.resize((200,200), Image.ANTIALIAS)
        img = ImageTk.PhotoImage(img)
        Panel = Label(window2, image = img)
        Panel.image = img
        Panel.place(x=250, y=250, anchor="center")
        
        if self.query.get() == "Brain Tumor Detection":
            preds = Brain_Tumor_predict(x)
            tv = ttk.Treeview(window2)
            style = ttk.Style(window2)
            style.configure('Treeview', rowheight=2)
            tv['columns']=('CATEGORY', 'TUMOR', 'NO TUMOR')
            tv.column('#0', width=0, stretch=NO)
            tv.column('CATEGORY', anchor=CENTER, width=80)
            tv.column('TUMOR', anchor=CENTER, width=80)
            tv.column('NO TUMOR', anchor=CENTER, width=80)

            tv.heading('#0', text='', anchor=CENTER)
            tv.heading('CATEGORY', text='CATEGORY', anchor=CENTER)
            tv.heading('TUMOR', text='TUMOR', anchor=CENTER)
            tv.heading('NO TUMOR', text='NO TUMOR', anchor=CENTER)

            tv.insert(parent='', index=0, iid=0, text='', values=('PROBABILITY',str(round(preds[0][1],2)),str(round(preds[0][0],2))))
            
            tv.place(x=250, y=450, anchor="center")
        
        elif self.query.get() == "Breast Cancer Detection": 
            preds = predict_img(x)
            tv = ttk.Treeview(window2)
            style = ttk.Style(window2)
            style.configure('Treeview', rowheight=2)
            tv['columns']=('CATEGORY', 'Benign d=1', 'Malignant d=1',
                          'Benign d=2', 'Malignant d=2')
            tv.column('#0', width=0, stretch=NO)
            tv.column('CATEGORY', anchor=CENTER, width=80)
            tv.column('Benign d=1', anchor=CENTER, width=80)
            tv.column('Malignant d=1', anchor=CENTER, width=85)
            tv.column('Benign d=2', anchor=CENTER, width=80)
            tv.column('Malignant d=2', anchor=CENTER, width=85)

            tv.heading('#0', text='', anchor=CENTER)
            tv.heading('CATEGORY', text='CATEGORY', anchor=CENTER)
            tv.heading('Benign d=1', text='Benign d=1', anchor=CENTER)
            tv.heading('Malignant d=1', text='Malignant d=1', anchor=CENTER)
            tv.heading('Benign d=2', text='Benign d=2', anchor=CENTER)
            tv.heading('Malignant d=2', text='Malignant d=2', anchor=CENTER)                                     
            tv.insert(parent='', index=0, iid=0, text='', values=('PROBABILITY',round(preds[0],3),
                                                                  round(preds[1],3),
                                                                  round(preds[2],3),
                                                                  round(preds[3],3)))
            tv.place(x=250, y=385, anchor="center")
            
            Tv = ttk.Treeview(window2)
            style = ttk.Style(window2)
            style.configure('Treeview', rowheight=2)
            Tv['columns']=('CATEGORY', 'Benign d=3', 'Malignant d=3',
                          'Benign d=4', 'Malignant d=4')
            Tv.column('#0', width=0, stretch=NO)
            Tv.column('CATEGORY', anchor=CENTER, width=80)
            Tv.column('Benign d=3', anchor=CENTER, width=80)
            Tv.column('Malignant d=3', anchor=CENTER, width=85)
            Tv.column('Benign d=4', anchor=CENTER, width=80)
            Tv.column('Malignant d=4', anchor=CENTER, width=85)

            Tv.heading('#0', text='', anchor=CENTER)
            Tv.heading('CATEGORY', text='CATEGORY', anchor=CENTER)
            Tv.heading('Benign d=3', text='Benign d=3', anchor=CENTER)
            Tv.heading('Malignant d=3', text='Malignant d=3', anchor=CENTER)
            Tv.heading('Benign d=4', text='Benign d=4', anchor=CENTER)
            Tv.heading('Malignant d=4', text='Malignant d=4', anchor=CENTER)                                 
            Tv.insert(parent='', index=0, iid=0, text='', values=('PROBABILITY',round(preds[4],3),
                                                                  round(preds[5],3),
                                                                  round(preds[6],3),
                                                                  round(preds[7],3)))
            Tv.place(x=250, y=450, anchor="center")
            
        elif self.query.get() == "Lung Cancer Detection":
            preds = lung_predict(x)
            tv = ttk.Treeview(window2)
            style = ttk.Style(window2)
            style.configure('Treeview', rowheight=2)
            tv['columns']=('CATEGORY', 'Adenocarcinoma',
                           'Large Carcinoma',
                           'Normal',
                           'Squamous')
            tv.column('#0', width=0, stretch=NO)
            tv.column('CATEGORY', anchor=CENTER, width=80)
            tv.column('Adenocarcinoma', anchor=CENTER, width=100)
            tv.column('Large Carcinoma', anchor=CENTER, width=100)
            tv.column('Normal', anchor=CENTER, width=80)
            tv.column('Squamous', anchor=CENTER, width=80)

            tv.heading('#0', text='', anchor=CENTER)
            tv.heading('CATEGORY', text='CATEGORY', anchor=CENTER)
            tv.heading('Adenocarcinoma', text='Adenocarcinoma', anchor=CENTER)
            tv.heading('Large Carcinoma', text='Large Carcinoma', anchor=CENTER)
            tv.heading('Normal', text='Normal', anchor=CENTER)
            tv.heading('Squamous', text='Squamous', anchor=CENTER)

            tv.insert(parent='', index=0, iid=0, text='', values=('PROBABILITY',
                                                                  str(round(preds[0][0],3)),
                                                                  str(round(preds[0][1],3)),
                                                                  str(round(preds[0][2],3)),
                                                                  str(round(preds[0][3],3))))
            
            tv.place(x=250, y=450, anchor="center")
            
        elif self.query.get() == "Chest Infection Detection":
            preds = chest_predict(x)
            tv = ttk.Treeview(window2)
            style = ttk.Style(window2)
            style.configure('Treeview', rowheight=2)
            tv['columns']=('CATEGORY', 
                           'Covid',
                           'Normal',
                           'Viral Pneumonia')
            tv.column('#0', width=0, stretch=NO)
            tv.column('CATEGORY', anchor=CENTER, width=80)
            tv.column('Covid', anchor=CENTER, width=100)
            tv.column('Normal', anchor=CENTER, width=100)
            tv.column('Viral Pneumonia', anchor=CENTER, width=80)
            
            tv.heading('#0', text='', anchor=CENTER)
            tv.heading('CATEGORY', text='CATEGORY', anchor=CENTER)
            tv.heading('Covid', text='Covid', anchor=CENTER)
            tv.heading('Normal', text='Normal', anchor=CENTER)
            tv.heading('Viral Pneumonia', text='Viral Pneumonia', anchor=CENTER)
            
            tv.insert(parent='', index=0, iid=0, text='', values=('PROBABILITY',
                                                                  str(round(preds[0][0],3)),
                                                                  str(round(preds[0][1],3)),
                                                                  str(round(preds[0][2],3))))
            
            tv.place(x=250, y=450, anchor="center")
        
        elif self.query.get() == "Alzheimers Detection":
            preds = alzheimers_predict(x)
            tv = ttk.Treeview(window2)
            style = ttk.Style(window2)
            style.configure('Treeview', rowheight=2)
            tv['columns']=('CATEGORY', 
                           'Mild',
                           'Moderate',
                           'Non-Dementia',
                           'Very Mild')
            tv.column('#0', width=0, stretch=NO)
            tv.column('CATEGORY', anchor=CENTER, width=80)
            tv.column('Mild', anchor=CENTER, width=100)
            tv.column('Moderate', anchor=CENTER, width=100)
            tv.column('Non-Dementia', anchor=CENTER, width=80)
            tv.column('Very Mild', anchor=CENTER, width=80)
            
            tv.heading('#0', text='', anchor=CENTER)
            tv.heading('CATEGORY', text='CATEGORY', anchor=CENTER)
            tv.heading('Mild', text='Mild', anchor=CENTER)
            tv.heading('Moderate', text='Moderate', anchor=CENTER)
            tv.heading('Non-Dementia', text='Non-Dementia', anchor=CENTER)
            tv.heading('Very Mild', text='Very Mild', anchor=CENTER)
            
            tv.insert(parent='', index=0, iid=0, text='', values=('PROBABILITY',
                                                                  str(round(preds[0][0],3)),
                                                                  str(round(preds[0][1],3)),
                                                                  str(round(preds[0][2],3)),
                                                                  str(round(preds[0][3],3))))
            
            tv.place(x=250, y=450, anchor="center")
        
        elif self.query.get() == "Knee arthritis Detection":
            preds = knee_predict(x)
            tv = ttk.Treeview(window2)
            style = ttk.Style(window2)
            style.configure('Treeview', rowheight=2)
            tv['columns']=('CATEGORY', 
                           'Normal Knee',
                           'Slight Damage',
                           'Noticable Damage')

            tv.column('#0', width=0, stretch=NO)
            tv.column('CATEGORY', anchor=CENTER, width=80)
            tv.column('Normal Knee', anchor=CENTER, width=100)
            tv.column('Slight Damage', anchor=CENTER, width=100)
            tv.column('Noticable Damage', anchor=CENTER, width=110)
            
            tv.heading('#0', text='', anchor=CENTER)
            tv.heading('CATEGORY', text='CATEGORY', anchor=CENTER)
            tv.heading('Normal Knee', text='Normal Knee', anchor=CENTER)
            tv.heading('Slight Damage', text='Slight Damage', anchor=CENTER)
            tv.heading('Noticable Damage', text='Noticable Damage', anchor=CENTER)
            
            tv.insert(parent='', index=0, iid=0, text='', values=('PROBABILITY',
                                                                  str(round(preds[0][0],3)),
                                                                  str(round(preds[0][1],3)),
                                                                  str(round(preds[0][2],3))))
            
            
            
            tv.place(x=250, y=385, anchor="center")
            
            Tv = ttk.Treeview(window2)
            style = ttk.Style(window2)
            style.configure('Treeview', rowheight=2)
            Tv['columns']=('CATEGORY', 'Concerning Damage','Badly Damaged')
            Tv.column('#0', width=0, stretch=NO)
            Tv.column('CATEGORY', anchor=CENTER, width=80)
            Tv.column('Concerning Damage', anchor=CENTER, width=100)
            Tv.column('Badly Damaged', anchor=CENTER, width=100)


            Tv.heading('#0', text='', anchor=CENTER)
            Tv.heading('Concerning Damage', text='Concerning Damage', anchor=CENTER)
            Tv.heading('Badly Damaged', text='Badly Damaged', anchor=CENTER)                              
            Tv.insert(parent='', index=0, iid=0, text='', values=('PROBABILITY',
                                                                  str(round(preds[0][3],3)),
                                                                  str(round(preds[0][4],3))))
            Tv.place(x=250, y=450, anchor="center")
            
        
    def mainScreen(self, Window): 
        self.query = StringVar()
        self.query2 = StringVar()
        
        Window.geometry("600x600")
        Window.resizable(0,0)
        Header = Label(text="MEDSMART X-RAY SOLUTIONS", font=("Arial", 22))
        Header.place(x=300, y=55, anchor="center")

        Type = Label(text="Choose a Scan Type: ", font=("Arial", 14))
        Type.place(x=200, y=125, anchor="center")

        Combo = ttk.Combobox(Window, width=25, textvariable=self.query, state="readonly")
        Combo['values'] = ("Please Select A Model",
                           "Brain Tumor Detection",
                           "Alzheimers Detection",
                           "Chest Infection Detection",
                           "Lung Cancer Detection", 
                           "Breast Cancer Detection",
                           "Knee arthritis Detection")
        Combo.current(0)
        Combo.place(x=425, y=125, anchor='center')
        
        def openfile():
            filename = filedialog.askopenfilename(title='"pen')
            return filename
        
        def displayImg():
            global x
            x = openfile()
            img = Image.open(x)
            img = img.resize((250,250), Image.ANTIALIAS)
            img = ImageTk.PhotoImage(img)
            panel = Label(Window, image = img)
            panel.image = img
            panel.place(x=300, y=325, anchor="center")
            

        Upload = Button(text="UPLOAD IMAGE", command=displayImg)
        Upload.place(x=300 ,y=180 ,anchor = 'center')

        Results = Button(text="DISPLAY RESULTS", command=self.SimpleDisplay) 
        Results.place(x=300, y=475, anchor="center")

        Report = Button(text="PRINT REPORT", command=self.ReportScreen)
        Report.place(x=300, y=525, anchor="center")
        

root = Tk()
app = Windows()
app.mainScreen(root)
root.mainloop()